# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [2]:
###
import open3d as o3d
import numpy as np
import copy
import collections


# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
# Add code here    

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def get_init_matches(feature_source, feature_target):
    ##对 feature target fpfh 建立 kd—tree
    fpfh_search_tree = o3d.geometry.KDTreeFlann(feature_target)
    ##建立 pairs
    _,N = feature_source.shape
    matches = []
    for i in range(N):
        query = feature_source[:,i]
        _, idx_nn_target, _ = fpfh_search_tree.search_knn_vector_xd(query, 1)   #source -> target
        matches.append([i,idx_nn_target[0]])    #通过knn 寻找唯一 的 nearest points 一一配对 构建pair

    matches = np.asarray(matches)
    return  matches
def global_registration_Ransac(source,target,feature_source,target_source):


In [10]:
source = o3d.io.read_point_cloud("r1.pcd")
target = o3d.io.read_point_cloud("r2.pcd")

# Used for downsampling.
voxel_size = 0.05
source_down=source.voxel_down_sample(voxel_size)
target_down=target.voxel_down_sample(voxel_size)

idx_target = 3440
idx_source = 4760

print(target_down)
# Show models side by side
#draw_registrations(source, target)
## 构建 kd-tree
search_tree_source = o3d.geometry.KDTreeFlann(source_down)
search_tree_target = o3d.geometry.KDTreeFlann(target_down)
#step3 iss 特征点提取
#keypoints_source = detect(pcd_source, search_tree_source, radius)
#keypoints_target = detect(pcd_target, search_tree_target, radius)
#step4 fpfh特征点描述 feature description
#pcd_source_keypoints = pcd_source.select_by_index(keypoints_source['id'].values)
##fpfh 进行 特征点描述
# at here ,I just use all the data as feature point
radius = 0.05


PointCloud with 3440 points.


In [ ]:
# find feature
source.estimate_normals()
target.estimate_normals()

print(type(source))
feature_source = o3d.pipelines.registration.compute_fpfh_feature(source_down, search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))
feature_target = o3d.pipelines.registration.compute_fpfh_feature(target_down, search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))

In [ ]:
#RANSAC configuration:
RANSACParams = collections.namedtuple(
    'RANSACParams',
    [
        'max_workers',
        'num_samples',
        'max_correspondence_distance', 'max_iteration', 'max_validation', 'max_refinement'
    ]
)
# fast pruning algorithm configuration:
CheckerParams = collections.namedtuple(
    'CheckerParams',
    ['max_correspondence_distance', 'max_edge_length_ratio', 'normal_angle_threshold']
)
# generate matches:
distance_threshold_init = 1.5 * radius
distance_threshold_final = 1.0 * radius

#step5 RANSAC Registration,初配准、得到初始旋转、平移矩阵
init_result  = global_registration_Ransac(
    idx_target,idx_source,
    source_down, target_down,
    feature_source, feature_target,
    ransac_params=RANSACParams(
        max_workers=5,
        num_samples=4,
        max_correspondence_distance=distance_threshold_init,
        max_iteration=100000,
        max_validation=500,
        max_refinement=30
    ),
    checker_params=CheckerParams(
        max_correspondence_distance=distance_threshold_init,
        max_edge_length_ratio=0.9,
        normal_angle_threshold=None
    )
)

